# CCNMF analysis of real paired scRNA and scDNA data from a cell mixture

### Download the [ovarian cancer cell lines datasets](https://zenodo.org/record/2363826#.XjkYohNKhE4) from [clonealign: statistical integration of independent single-cell RNA and DNA sequencing data from human cancers.](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1645-z) 

In [2]:
# load necessary packages 
library(NMF)
library(dplyr)
library(tidyr)

library(Matrix)
library(devtools)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(org.Hs.eg.db)

library(pheatmap)
library(ggplot2)
library(Seurat)
library(stringr)
library(Rtsne)
library(uwot)
library(mclust)
library(biomaRt)
library(cowplot)
library(ggplotify)
library(RColorBrewer)
library(minerva)

library(CCNMF)

In [3]:
# Input the single-cell RNA-seq (gene expression) data and single-cell DNA-seq (copy number variants) data.

pathDNA <- '~/Desktop/CCNMF-master/Extra_code/clonealign-processed-data/T_OV2295/cnv'
pathRNA_TOV <- '~/Desktop/CCNMF-master/Extra_code/clonealign-processed-data/T_OV2295/10X/TOV2295/outs/filtered_gene_bc_matrices/hg19'
pathRNA_OV <-'~/Desktop/CCNMF-master/Extra_code/clonealign-processed-data/T_OV2295/10X/OV2295n2/outs/filtered_gene_bc_matrices/hg19'

# Read All CNV data
OV_CNV <- read.csv(file.path(pathDNA, 'cnv_data.csv'))

# Read the genes of scRNA-seq data
TOV_gene <- read.table(file.path(pathRNA_TOV, 'genes.tsv'))
OV_gene <- read.table(file.path(pathRNA_OV, 'genes.tsv'))

# Load single-cell TOV data includes 4918 cells with 32738 genes 
TOV_RNA <- InputRNA(pathRNA_TOV)

# Load single-cell OV data includes 1717 cells with 32738 genes 
OV_RNA <- InputRNA(pathRNA_OV)

# Find the common barcodes between OV and TOV scRNA datasets 
commonBarcodes <- intersect(colnames(OV_RNA), colnames(TOV_RNA))
index <- matrix(0, 1, length(commonBarcodes))
for (i in 1:length(commonBarcodes)){
  index[i] <- which(colnames(TOV_RNA) == commonBarcodes[i])
}

# Change the barcodes's name in TOV RNA-seq data
colnames(TOV_RNA)[index] = paste0(commonBarcodes, 'T')

###Combine these two datasets as a RNA matrix
AllRNA <- cbind(TOV_RNA, OV_RNA)


In [4]:
# Function for handling copy number data to default input 
ProcessOriginalData <- function(Data){
  singlecellid <- Data$single_cell_id
  Outputdata <- data.frame(
    chr=Data$chr[which(singlecellid == singlecellid[1])],
    start=Data$start[which(singlecellid == singlecellid[1])],
    end=Data$end[which(singlecellid == singlecellid[1])]
    )
  Outputdata$width <- Outputdata$end - Outputdata$start
  N <- length(which(singlecellid == singlecellid[1]))
  for (i in 1: length(unique(Data$single_cell_id))){
    Cell <- Data$copy_number[as.integer((i-1)*N + 1) : as.integer(i * N)]
    Outputdata <- cbind(Outputdata,Cell)
  }
  #cell_number <- length(unique(Data$single_cell_id))
  colnames(Outputdata)[5: dim(Outputdata)[2]] <- levels(unique(Data$single_cell_id)[1])
  return(Outputdata)
}

In [5]:
# Preprocessing the scRNA-seq data by Seurat including normalization, scaling and selecting high variable genes.   
RNAobject <- run_Seurat_RNA(AllRNA)

## Convert copy number variants data as correct input format and match the same genes with the seleced high-variable genes in scRNA-seq data.
CNVmatrix <- ProcessOriginalData(OV_CNV)

#Then we use a sigmoid fucnction transform the negative elements in scaled RNA matrix to non-negative 
#since the CCNMF needs non-negative matrices as input.
sigmoid <- function(x){1/(1+ exp(-x))}
RNAmatrix1 <- RNAobject@assays$RNA@scale.data[RNAobject@assays$RNA@meta.features$vst.variable, ]
replaceindex <- which(RNAmatrix1 < 0 )
RNAmatrix2 <- RNAmatrix1
RNAmatrix2[replaceindex] <- sigmoid(RNAmatrix1)[replaceindex]

Centering and scaling data matrix

PC_ 1 
Positive:  ENSG00000140988, ENSG00000167996, ENSG00000115457, ENSG00000143320, ENSG00000106211, ENSG00000160678, ENSG00000165215, ENSG00000135480, ENSG00000101443, ENSG00000137309 
	   ENSG00000125968, ENSG00000113946, ENSG00000167969, ENSG00000161798, ENSG00000198467, ENSG00000172216, ENSG00000100906, ENSG00000137962, ENSG00000114638, ENSG00000141401 
	   ENSG00000167755, ENSG00000188042, ENSG00000146038, ENSG00000184897, ENSG00000171223, ENSG00000168309, ENSG00000173917, ENSG00000103260, ENSG00000196154, ENSG00000135205 
Negative:  ENSG00000165949, ENSG00000130303, ENSG00000173926, ENSG00000126709, ENSG00000071282, ENSG00000160932, ENSG00000143631, ENSG00000142676, ENSG00000008394, ENSG00000073803 
	   ENSG00000090273, ENSG00000179761, ENSG00000100028, ENSG00000148677, ENSG00000187608, ENSG00000181991, ENSG00000140612, ENSG00000169504, ENSG00000188313, ENSG00000075624 
	   ENSG00000169641, ENSG00000182899, ENSG00000185339, ENSG00000169612, EN

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 6635
Number of edges: 213261

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9204
Number of communities: 5
Elapsed time: 0 seconds


In [6]:
# Estimate the corespponding gene region and chromosome bins by reference genome 'hg19', meanwhile, construct a identify matrix as the coupling matrix.

InterVariable <- Estimate_Coupled_matrix(RNAmatrix2, CNVmatrix, reference_name = 'hg19')

Joining, by = "ensembl_gene_id"

Warning message:
“Column `ensembl_gene_id` has different attributes on LHS and RHS of join”


In [7]:
#Convert the Ensembl Symbol genes to HUGO genes for processed DNA and RNA matrices 
CNVmatrix <- InterVariable[[1]]
RNAmatrix <- AllRNA[rownames(CNVmatrix), ]
RNAscale <- RNAmatrix2[rownames(CNVmatrix), ]
CoupledMatrix <- InterVariable[[3]]

Gene <- read.table(file.path(pathRNA_TOV, 'genes.tsv'))
Genename <- ConvertGenenames(rownames(CNVmatrix), Gene, Logic = FALSE)
rownames(CNVmatrix) <- Genename
rownames(RNAmatrix) <- Genename
rownames(RNAscale) <- Genename

In [8]:
## Run CCNMF for paired scDNA and scRNA datasets
ResultsCCNMF <- run_CCNMF(ncluster = 2, CNVmatrix, RNAscale, CoupledMatrix, lambda1=1, lambda2=1, mu=1)
S1 <- ResultsCCNMF[[5]]
S2 <- ResultsCCNMF[[6]]

[1] "Initializing NMF for CNV matrirx..."
[1] "Initializing NMF for scRNA matrirx..."
[1] "Initializing the parameters lambda1, lambda2 and mu..."
[1] "Start Coupled NMF"
[1] "Iterating coupledNMF..."
[1] "eq1 1940235"
[1] "eq2 4681809"
[1] "eq3 39635"
[1] "eq4 107728"
[1] "eq5 6622043"
[1] "Run time:366.008seconds"


In [9]:
## Find the differential genes between clusters based on scRNA data
RNADE <- DiffExp(RNAmatrix, S2)

In [ ]:
#Output the heatmap and tsne figures of CCNMF, the main figure's pdf can be saved in current path.
PlotMainResult(DNAmatrix, RNAscale, ResultsCCNMF, RNADE)

![](realdata_figure.png)